In [2]:
import sys
from dotenv import load_dotenv
import os

load_dotenv()

sys.path.append(os.getenv("ROOT"))

In [3]:
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

from silverfund.backtester import Backtester
from silverfund.datasets.master_monthly import MasterMonthly
from silverfund.enums import Interval
from silverfund.optimizers.new_constraints import *
from silverfund.research.betting_against_beta.barra_beta.barra_beta_strategy import BarraBetaStrategy

In [9]:
# Daily backtest
start_date = date(2022, 1, 1)
end_date = date(2022, 12, 1)

# Load historical dataset
historical_data = (
    MasterMonthly(
        start_date=start_date,
        end_date=end_date,
    )
    .load_all()
)

In [10]:
historical_data

date,barrid,date_lag_1d,date_lag_2d,date_lag_1m,date_lag_2m,currency,mktcap,price,logret,ret,div_yield,total_risk,spec_risk,histbeta,predbeta,log_spec_ret,spec_ret
date,str,date,date,date,date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2022-01-31,"""USA06Z1""",2022-01-28,2022-01-27,2021-12-31,2021-11-30,"""USD""",5.5624e8,4.96,-0.197,-0.178809,0.0,0.820023,0.731666,1.297365,1.337664,0.007194,0.00722
2022-02-28,"""USA06Z1""",2022-02-25,2022-02-24,2022-01-31,2021-12-31,"""USD""",5.6633e8,5.05,0.017985,0.018148,0.0,0.8317529,0.7126182,1.379646,1.502205,0.084383,0.088046
2022-03-31,"""USA06Z1""",2022-03-30,2022-03-29,2022-02-28,2022-01-31,"""USD""",5.2921e8,4.71,-0.069699,-0.067325,0.0,0.791204,0.666928,1.450875,1.542893,-0.073202,-0.070587
2022-04-29,"""USA06Z1""",2022-04-28,2022-04-27,2022-03-31,2022-02-28,"""USD""",4.4774e8,3.95,-0.175975,-0.161361,0.0,0.773508,0.644653,1.380664,1.533729,0.03746,0.038171
2022-05-31,"""USA06Z1""",2022-05-27,2022-05-26,2022-04-29,2022-03-31,"""USD""",4.4321e8,3.91,-0.010177,-0.010126,0.0,0.768461,0.628478,1.481874,1.576411,0.031012,0.031498
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022-11-30,"""USBOFX1""",2022-11-29,2022-11-28,2022-10-31,2022-09-30,"""USD""",1.0783e9,15.82,-0.04632,-0.045264,0.0,0.809521,0.67956,0.873843,1.281963,-0.003122,-0.003117
2022-10-31,"""USBOGD1""",2022-10-28,2022-10-27,2022-09-30,2022-08-31,"""USD""",9.7026e7,25.3,0.222651,0.249384,null,0.405745,0.287345,0.83941,0.990002,0.164542,0.178853
2022-11-30,"""USBOGD1""",2022-11-29,2022-11-28,2022-10-31,2022-09-30,"""USD""",9.4035e7,24.52,-0.031317,-0.030831,null,0.440057,0.298054,0.827399,1.118619,-0.042988,-0.042077


In [11]:
# Create backtest instance
bt = Backtester(
    start_date=start_date,
    end_date=end_date,
    interval=Interval.MONTHLY,
    historical_data=historical_data,
    strategy=BarraBetaStrategy,
    constraints=[FullInvestment, NoBuyingOnMargin, ShortingLimit, UnitBeta],
    security_identifier="barrid"
)

In [12]:
# Run backtest
pnl = bt.run()

Running strategy: 100%|██████████| 11/11 [04:51<00:00, 26.50s/it]


In [13]:
# Table
print("\n" + "-" * 50 + " Backtest P&L " + "-" * 50)
print(pnl)


-------------------------------------------------- Backtest P&L --------------------------------------------------
shape: (11, 6)
┌────────────┬──────────┬───────────────┬──────────────────┬─────────┬────────┐
│ date       ┆ n_assets ┆ portfolio_ret ┆ portfolio_logret ┆ cumprod ┆ cumsum │
│ ---        ┆ ---      ┆ ---           ┆ ---              ┆ ---     ┆ ---    │
│ date       ┆ u32      ┆ f64           ┆ f64              ┆ f64     ┆ f64    │
╞════════════╪══════════╪═══════════════╪══════════════════╪═════════╪════════╡
│ 2022-01-31 ┆ 3047     ┆ NaN           ┆ NaN              ┆ NaN     ┆ NaN    │
│ 2022-02-28 ┆ 3037     ┆ NaN           ┆ NaN              ┆ NaN     ┆ NaN    │
│ 2022-03-31 ┆ 3028     ┆ NaN           ┆ NaN              ┆ NaN     ┆ NaN    │
│ 2022-04-29 ┆ 3013     ┆ NaN           ┆ NaN              ┆ NaN     ┆ NaN    │
│ 2022-05-31 ┆ 3001     ┆ NaN           ┆ NaN              ┆ NaN     ┆ NaN    │
│ …          ┆ …        ┆ …             ┆ …                ┆ …       